In [1]:
import pandas as pd
import numpy as np
import copy
import re
from datetime import date
import warnings
warnings.filterwarnings('ignore')

In [2]:
## Import data set: 
## *** Important: Use the cleaned dataset, or run 'csv_cleaning.csv' on raw data before proceeding (as it creates 'UniqueID' for individual participants)!! ***
df = pd.read_csv('YSET_Dec2008_Dec2019_cleaned20190531.csv')

In [3]:
## Creating new dataframe with UniqueID and Responses for Question 1 through 56 (Section A through Section IJ)
df_question = copy.deepcopy(df.loc[:,"a1_combo":"ij56_ever_combo"])
df_question['y_Flag_GangInvolvement'] = copy.deepcopy(df['y_Flag_GangInvolvement'])
df_question['YSET_I_R'] = copy.deepcopy(df['YSET_I_R'])
df_question['YSETResults'] = copy.deepcopy(df['YSETResults'])
df_question['y_TotalFactors'] = copy.deepcopy(df['y_TotalFactors'])
df_question['Age'] = copy.deepcopy(df['Age'])
df_question['Date'] = copy.deepcopy(df['y_DateS1_Interview'])
df_question['Gender'] = copy.deepcopy(df['Gender'])
df_question = df_question.join(copy.deepcopy(df.loc[:,'GRYD_Zone':'Ethnicity_Other']))
df_question['UniqueID'] = copy.deepcopy(df['UniqueID'])

cols = df_question.columns.tolist()
cols = cols[-1:] + cols[:-1]
df_question = df_question[cols]

In [4]:
## Col: 'UniqueID'
df_question['UniqueID'] = pd.to_numeric(df_question['UniqueID'], downcast = 'integer', errors = 'coerce')


In [5]:
## Col: 'y_Flag_GangInvolvement'
df_question['y_Flag_GangInvolvement'] = pd.to_numeric(df_question['y_Flag_GangInvolvement'], downcast = 'integer', errors = 'coerce')

In [6]:
## Col: 'Date'
df_question['Date'] =  pd.to_datetime(df_question['Date'])
invalid_date = df_question[df_question['Date'] > pd.to_datetime(date.today())].index.tolist()
for index in invalid_date:
    df_question.loc[index, 'Date'] = np.nan

In [7]:
## Col: 'YSET_I_R'
for i in list(df_question[df_question.YSET_I_R == 1].index):
    df_question.at[i, 'YSET_I_R'] *= 10
    
uniIDlist = list(df_question.UniqueID.value_counts().index)
for id in uniIDlist:
    if len(df_question[df_question.UniqueID == id][df_question[df_question.UniqueID == id].YSET_I_R == 2]) > 0:
        for i in df_question[df_question.UniqueID == id][df_question[df_question.UniqueID == id].Date >= df_question[df_question.UniqueID == id][df_question[df_question.UniqueID == id].YSET_I_R == 2].Date.values[0]].YSET_I_R.index:
            if df_question.at[i, 'YSET_I_R'] == 2: df_question.at[i, 'YSET_I_R'] *= 10
            else: df_question.at[i, 'YSET_I_R'] += 10
    if len(df_question[df_question.UniqueID == id][df_question[df_question.UniqueID == id].YSET_I_R == 3]) > 0:
        for i in df_question[df_question.UniqueID == id][df_question[df_question.UniqueID == id].Date >= df_question[df_question.UniqueID == id][df_question[df_question.UniqueID == id].YSET_I_R == 3].Date.values[0]].YSET_I_R.index:
            if df_question.at[i, 'YSET_I_R'] == 3: df_question.at[i, 'YSET_I_R'] *= 10
            else: df_question.at[i, 'YSET_I_R'] += 20
    if len(df_question[df_question.UniqueID == id][df_question[df_question.UniqueID == id].YSET_I_R == 4]) > 0:
        for i in df_question[df_question.UniqueID == id][df_question[df_question.UniqueID == id].Date >= df_question[df_question.UniqueID == id][df_question[df_question.UniqueID == id].YSET_I_R == 4].Date.values[0]].YSET_I_R.index:
            if df_question.at[i, 'YSET_I_R'] == 4: df_question.at[i, 'YSET_I_R'] *= 10
            else: df_question.at[i, 'YSET_I_R'] += 30


In [8]:
## Col: 'y_TotalFactors'
df_question['y_TotalFactors'] = pd.to_numeric(df_question['y_TotalFactors'], downcast = 'integer', errors = 'coerce')
df_question.loc[(df_question.y_TotalFactors == 88), 'y_TotalFactors'] = np.nan

In [9]:
## Col: 'Age'
df_question['Age'] = pd.to_numeric(df_question['Age'], downcast = 'integer', errors = 'coerce')
df_question.loc[(df_question.Age == 99), 'Age'] = np.nan

In [10]:
## Col: 'Gender': 1 Male, 2 Female
df_question['Gender'] = pd.to_numeric(df_question['Gender'], downcast = 'integer', errors = 'coerce')
df_question.loc[(df_question.Gender == -1), 'Gender'] = np.nan
df_question.loc[(df_question.Gender == 99), 'Gender'] = np.nan

In [11]:
## Col: 'GRYD_Zone'
df_question['GRYD_Zone'] = pd.to_numeric(df_question['GRYD_Zone'], downcast = 'integer', errors = 'coerce')
df_question.loc[(df_question.GRYD_Zone == 99), 'GRYD_Zone'] = np.nan

In [12]:
## Col: 'Ethnicity_Asian':'Ethnicity_Other'
for col in df_question.loc[:,'Ethnicity_Asian':'Ethnicity_Other'].columns.tolist():
    
    df_question[col] = pd.to_numeric(df_question[col], downcast = 'integer', errors = 'coerce')
    df_question.loc[(df_question[col] == 99), col] = np.nan
    df_question.loc[(df_question[col] == 9), col] = np.nan
    df_question.loc[(df_question[col] == 11), col] = np.nan

In [13]:
## Col: 'ij40_6mos_combo':'ij56_ever_combo'
for col in df_question.loc[:,'ij40_6mos_combo':'ij56_ever_combo'].columns.tolist():
    
    df_question[col] = pd.to_numeric(df_question[col], downcast = 'integer', errors = 'coerce')
    df_question.loc[(df_question[col] == -1), col] = np.nan
    df_question.loc[(df_question[col] == 11), col] = np.nan
    df_question.loc[(df_question[col] == 99), col] = np.nan
    df_question.loc[(df_question[col] == 9), col] = np.nan

In [14]:
# checking IJ consistency
for var in range(40,56):
    df_question.loc[np.where((df_question['ij{}_6mos_combo'.format(var)] ==1 ) & (df_question['ij{}_ever_combo'.format(var)]== 0), 1, 0), 'ij{}_ever_combo'.format(var)] = np.nan

In [15]:
## Col: 't38_combo', 't39_combo'
for col in df_question.loc[:,'t38_combo':'t39_combo'].columns.tolist():
    
    df_question[col] = pd.to_numeric(df_question[col], downcast = 'integer', errors = 'coerce')

    df_question.loc[(df_question[col] == 99), col] = np.nan
    df_question.loc[(df_question[col] == 999), col] = np.nan
    
    df_question.loc[(df_question[col] > 4), col] = 4

In [16]:
## All the rest columns:    
for col in df_question.loc[:,'a1_combo':'h37_combo'].columns.tolist():
    
    df_question[col] = pd.to_numeric(df_question[col], downcast = 'integer', errors = 'coerce')
    df_question.loc[(df_question[col] == -1), col] = np.nan
    df_question.loc[(df_question[col] == 11), col] = np.nan
    df_question.loc[(df_question[col] == 99), col] = np.nan
    df_question.loc[(df_question[col] == 9), col] = np.nan
    
    df_question.loc[(df_question[col] % 1 != 0), col] = np.nan
    
    df_question.loc[(df_question[col] > 5), col] = np.nan
    
    if max(df_question[col]) == 5: 
        df_question.loc[(df_question[col] == 0), col] = np.nan

In [17]:
## Section A through H
prev_sect = ''
for name in df_question.loc[:,"a1_combo":"h37_combo"].columns.tolist():
    m = re.search("\d", name)
    curr_sect = name[:re.search("\d", name).start()]
    if curr_sect == prev_sect:
        df_question['Section_{}'.format(prev_sect)] += copy.deepcopy(df_question[name])
    else:
        df_question['Section_{}'.format(curr_sect)] = copy.deepcopy(df_question[name])
        prev_sect = curr_sect

In [18]:
## Section T
def sectionT(row):
    if row.t38_combo < 2 and row.t39_combo < 2:
        return 0
    elif row.t38_combo >= 2 and row.t39_combo >= 2:
        return 2
    else: return 1
df_question['Section_t'] = df_question.apply(sectionT, axis = 1)

In [19]:
## Section IJ: 6mos
df_question['Section_ij_6mos'] = copy.deepcopy(df_question["ij40_6mos_combo"])
for col in df_question.loc[:,"ij41_6mos_combo":"ij56_6mos_combo"].columns.tolist():
    df_question['Section_ij_6mos'] += copy.deepcopy(df_question[col])

In [21]:
## Section IJ: ever
df_question['Section_ij_ever'] = copy.deepcopy(df_question["ij40_ever_combo"])
for col in df_question.loc[:,"ij41_ever_combo":"ij56_ever_combo"].columns.tolist():
    df_question['Section_ij_ever'] += copy.deepcopy(df_question[col]) 

df_question.to_csv('YSET_Dec08_Dec19_a1_ij56ever_cleaned.csv')